In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3 , padding=1)
        self.norm1 = nn.BatchNorm2d(32)
        self.drop1 = nn.Dropout2d(0.05)

        self.conv2 = nn.Conv2d(32, 32, 3)
        self.norm2 = nn.BatchNorm2d(32)
        self.drop2 = nn.Dropout2d(0.05)
        self.pool2 = nn.MaxPool2d(2, 2) 

        self.conv3 = nn.Conv2d(32, 16, 3)
        self.norm3 = nn.BatchNorm2d(16)
        self.drop3 = nn.Dropout2d(0.05)


        self.conv4 = nn.Conv2d(16, 16, 3)
        self.norm4 = nn.BatchNorm2d(16)
        self.drop4 = nn.Dropout2d(0.05)
        self.pool4 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(16, 16, 3)
        self.pool5 = nn.AvgPool2d(2)
        self.conv7 = nn.Linear(16*1*1, 10)

    def forward(self, x):
        x = self.drop1(self.norm1(F.relu(self.conv1(x))))
        x = self.pool2(self.drop2(self.norm2(F.relu(self.conv2(x)))))
        x = self.drop3(self.norm3(F.relu(self.conv3(x))))
        x = self.pool4(self.drop4(self.norm4(F.relu(self.conv4(x)))))
        x = self.pool5(self.conv5(x))
        x = x.view(-1, 16*1*1)
        x = self.conv7(x)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
         Dropout2d-3           [-1, 32, 28, 28]               0
            Conv2d-4           [-1, 32, 26, 26]           9,248
       BatchNorm2d-5           [-1, 32, 26, 26]              64
         Dropout2d-6           [-1, 32, 26, 26]               0
         MaxPool2d-7           [-1, 32, 13, 13]               0
            Conv2d-8           [-1, 16, 11, 11]           4,624
       BatchNorm2d-9           [-1, 16, 11, 11]              32
        Dropout2d-10           [-1, 16, 11, 11]               0
           Conv2d-11             [-1, 16, 9, 9]           2,320
      BatchNorm2d-12             [-1, 16, 9, 9]              32
    

<ipython-input-2-e6f68e56fb19>:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 16):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-e6f68e56fb19>:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.11858775466680527 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.11it/s]



Test set: Average loss: 0.0491, Accuracy: 9853/10000 (98.53%)



loss=0.047411784529685974 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.79it/s]



Test set: Average loss: 0.0383, Accuracy: 9869/10000 (98.69%)



loss=0.16353555023670197 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 31.34it/s]


...


Test set: Average loss: 0.0256, Accuracy: 9934/10000 (99.34%)



loss=0.009994844906032085 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.41it/s]



Test set: Average loss: 0.0247, Accuracy: 9920/10000 (99.20%)



loss=0.015913566574454308 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.78it/s]



Test set: Average loss: 0.0251, Accuracy: 9930/10000 (99.30%)

